In [12]:
# Imports
import pandas as pd
import numpy as np

# Affichage
pd.set_option("display.max_columns", None)

# Chargement
df = pd.read_csv(
    "../data/raw/transport_incidents.csv",
    parse_dates=["EventStartTime", "EventEndTime"]
)

# Aperçu
df.head()


,EventStartTime,EventEndTime,EventDuration(min),TransportType,Line,Status,IncidentCause,IncidentSummary,DetailedInfo,FullDescription
0,2024-04-22 07:52:00,2024-04-22 08:30:00,38.0,Trains,A,Works,De 22h40 à 01h41 : Travaux 3 h Travaux,NaN,NaN,De 22h40 à 01h41 : Travaux 3 h Travaux | 01h41...
1,2024-04-23 04:28:00,2024-04-23 04:36:00,8.0,Trains,A,Disruption,De 05h16 à 07h00 : Prolongation des travaux ef...,NaN,NaN,De 05h16 à 07h00 : Prolongation des travaux ef...
2,2024-04-23 04:36:00,2024-04-23 04:54:00,18.0,Trains,A,Disruption,NaN,NaN,NaN,NaN
3,2024-04-23 05:16:00,2024-04-23 05:20:00,4.0,Trains,A,Disruption,De 07h54 à 09h36 : Affaires oubliées 101 min A...,NaN,NaN,De 07h54 à 09h36 : Affaires oubliées 101 min A...
4,2024-04-23 05:20:00,2024-04-23 05:22:00,2.0,Trains,A,Interruption,De 07h54 à 09h36 : Affaires oubliées 101 min A...,NaN,NaN,De 07h54 à 09h36 : Affaires oubliées 101 min A...


In [13]:
# Infos générales
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52453 entries, 0 to 52452
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   EventStartTime      52453 non-null  datetime64[ns]
 1   EventEndTime        52453 non-null  datetime64[ns]
 2   EventDuration(min)  52453 non-null  float64       
 3   TransportType       52453 non-null  object        
 4   Line                52453 non-null  object        
 5   Status              52453 non-null  object        
 6   IncidentCause       48683 non-null  object        
 7   IncidentSummary     15761 non-null  object        
 8   DetailedInfo        3921 non-null   object        
 9   FullDescription     48683 non-null  object        
dtypes: datetime64[ns](2), float64(1), object(7)
memory usage: 4.0+ MB


In [15]:
# Renommer la cible
df = df.rename(columns={
    "EventDuration(min)": "delay_minutes"
})

# Suppression valeurs invalides
df = df.dropna(subset=["delay_minutes"])
df = df[df["delay_minutes"] >= 0]

# Variables temporelles
df["hour"] = df["EventStartTime"].dt.hour
df["dayofweek"] = df["EventStartTime"].dt.dayofweek
df["month"] = df["EventStartTime"].dt.month

# Heure de pointe
df["heure_pointe"] = df["hour"].isin([7,8,9,16,17,18,19]).astype(int)

# Incident majeur
df["incident_majeur"] = df["Status"].isin(
    ["Disruption", "Works"]
).astype(int)

df.head()

,EventStartTime,EventEndTime,delay_minutes,TransportType,Line,Status,IncidentCause,IncidentSummary,DetailedInfo,FullDescription,hour,dayofweek,month,heure_pointe,incident_majeur
0,2024-04-22 07:52:00,2024-04-22 08:30:00,38.0,Trains,A,Works,De 22h40 à 01h41 : Travaux 3 h Travaux,NaN,NaN,De 22h40 à 01h41 : Travaux 3 h Travaux | 01h41...,7,0,4,1,1
1,2024-04-23 04:28:00,2024-04-23 04:36:00,8.0,Trains,A,Disruption,De 05h16 à 07h00 : Prolongation des travaux ef...,NaN,NaN,De 05h16 à 07h00 : Prolongation des travaux ef...,4,1,4,0,1
2,2024-04-23 04:36:00,2024-04-23 04:54:00,18.0,Trains,A,Disruption,NaN,NaN,NaN,NaN,4,1,4,0,1
3,2024-04-23 05:16:00,2024-04-23 05:20:00,4.0,Trains,A,Disruption,De 07h54 à 09h36 : Affaires oubliées 101 min A...,NaN,NaN,De 07h54 à 09h36 : Affaires oubliées 101 min A...,5,1,4,0,1
4,2024-04-23 05:20:00,2024-04-23 05:22:00,2.0,Trains,A,Interruption,De 07h54 à 09h36 : Affaires oubliées 101 min A...,NaN,NaN,De 07h54 à 09h36 : Affaires oubliées 101 min A...,5,1,4,0,0


In [16]:
# Sauvegarde
df.to_csv("../data/processed/clean_data.csv", index=False)
print("Données nettoyées sauvegardées.")


Données nettoyées sauvegardées.
